In [8]:
from src.data import ModuleData
datas = ModuleData.load([] , ['std_lag1_10' , 'std_lag1_20' , 'res_lag1_20'] , fit = False , predict = True)

Load  1 DataBlocks...... finished! Cost 0.08 secs
Align 1 DataBlocks...... finished! Cost 0.16 secs


In [11]:
import torch
a = torch.load('./data/Interim/DataBlock/Y.00.pt')

In [1]:
%run src_runs/autorun/daily_update.py

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!


In [6]:
from src.api import DataAPI , ModelAPI
DataAPI.update()
ModelAPI.update()

--------------------------------------------------------------------------------
***************************** PREPARE PREDICT DATA *****************************
predict is True , Data Processing start!
6 datas : ['y', 'day', '30m', 'style', 'indus', 'week'] , from -366 to None
y blocks loading start!
 --> labels_ts blocks reading [ret10_lag] DataBase...... finished! Cost 1.38 secs
 --> labels_ts blocks reading [ret20_lag] DataBase...... finished! Cost 1.72 secs
 --> labels_ts blocks merging (2)...... finished! Cost 0.32 secs
 --> models blocks reading [tushare_cne5_exp] DataBase...... finished! Cost 5.79 secs
y blocks loading finished! Cost 10.00 secs
y blocks process...... finished! Cost 10.02 secs
y blocks masking...... finished! Cost 0.09 secs
y blocks saving ...... finished! Cost 0.40 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 21.02 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> tra

100%|██████████| 316/316 [00:00<00:00, 1501.64it/s]


  -->  Finish updating hidden feature extraction for HiddenExtractingModel(hidden_name=gru_day_V0,name=gru_day,submodels=['best'],nums=None) , len=2
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
******************************** UPDATE FACTORS ********************************
model_name is None, update all registered models (len=3)
  -->  Finish updating model prediction for RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0) , len=1
  -->  Finish updating model prediction for RegisteredModel(pred_name=gruRTN_day_V0,name=gruRTN_day,submodel=swalast,num=0) , len=1
  -->  Finish updating model prediction for RegisteredModel(pred_name=gru_day_V1,name=gru_avg,submodel=swalast,num=all) , len=1
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
***

In [5]:
import pandas as pd
import numpy as np
from src.basic import CALENDAR
from src.data import DATAVENDOR

date = 20241101
DATAVENDOR.INDI.acc('longdeb_to_debt' , date , 1)['longdeb_to_debt']


secid
1             NaN
2         23.6045
4         11.6913
6         38.5851
7         34.4723
           ...   
920060     2.5784
920066     0.0976
920088     9.3508
920099    14.8430
920118     3.1372
Name: longdeb_to_debt, Length: 5385, dtype: float64

In [7]:
DATAVENDOR.TRADE.collections['val'].data_frames.keys()

dict_keys([20240827, 20240828, 20240829, 20240830, 20240902, 20240903, 20240904, 20240905, 20240906, 20240909, 20240910, 20240911, 20240912, 20240913, 20240918, 20240919, 20240920, 20240923, 20240924, 20240925, 20240926, 20240927, 20240930, 20241008, 20241009, 20241010, 20241011, 20241014, 20241015, 20241016, 20241017, 20241018, 20241021, 20241022, 20241023, 20241024, 20241025, 20241028, 20241029, 20241030, 20241031, 20241101, 20241104, 20241105, 20241106, 20241107, 20241108, 20241111, 20241112, 20241113, 20241114, 20241115, 20241118, 20241119, 20241120, 20241121, 20241122, 20241125, 20241126, 20241127, 20241128, 20241129, 20241202, 20240729, 20240730, 20240731, 20240801, 20240802, 20240805, 20240806, 20240807, 20240808, 20240809, 20240812, 20240813, 20240814, 20240815, 20240816, 20240819, 20240820, 20240821, 20240822, 20240823, 20240826])

In [1]:
from src.basic import RegisteredModel
model =RegisteredModel.SelectModels('gru_day_V0')[0]
model.fmp_target_dates

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!


array([20241101, 20241108, 20241115, 20241122, 20241129])

In [1]:
from src.api import ModelAPI
model = ModelAPI.Testor('ts_mixer')
model.try_forward()
model.try_metrics()

D:\Coding\learndl\learndl\configs\nn\ts_mixer.yaml does not exist, trying default.yaml
Load  2 DataBlocks...... finished! Cost 0.09 secs
Align 2 DataBlocks...... finished! Cost 0.23 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 212/212 [00:00<?, ?it/s]


x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
score function of [spearman] calculated and success!
loss function of [ccc] calculated and success!
metrics :  Metrics(loss=ccc,metric=spearman,penalty=['hidden_corr'])
Test Metrics Success


ModelTestor(model=NNPredictor(model_full_name=ts_mixer_day_ShortTest@0@0@0)) , check [.model][.batch_data][.metrics]

In [1]:
from src.basic import RegisteredModel , CONF
from src.model.model_module.application import ModelPortfolioBuilder

model_name = 'gru_day_V0'

model = RegisteredModel.SelectModels(model_name)[0]
md = ModelPortfolioBuilder(model)
md

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!


ModelPortfolioBuilder(RegisteredModel(pred_name=gru_day_V0,name=gru_day,submodel=swalast,num=0))

In [3]:
from src.basic import CALENDAR
from src.factor.fmp import PortfolioBuilder
from src.factor.util import Portfolio
import pandas as pd
import numpy as np
md.reg_model.pred_dates
self = md
date = 20241101

resume = True

fmp_names = list(self.iter_fmp_names())

last_date = min(list(self.last_account_date().values()))
account_dates = [date for date in self.reg_model.fmp_dates if date > last_date] if resume else self.reg_model.fmp_dates
if len(account_dates) > 0:
    all_fmp_dfs = pd.concat([self.reg_model.load_fmp(date) for date in account_dates])

    account_dir = CONF.PATH.fmp_account.joinpath(self.reg_model.pred_name)
    account_dir.mkdir(parents=True , exist_ok=True)

    alpha_model = self.alpha_model(account_dates)

    for fmp_name in fmp_names:
        portfolio    = Portfolio.from_dataframe(all_fmp_dfs[all_fmp_dfs['name'] == fmp_name])
        account_path = account_dir.joinpath(f'{fmp_name}.pkl')
        builder = PortfolioBuilder.from_full_name(fmp_name , alpha_model , portfolio)
        builder.accounting(account_path=account_path,resume=resume,deploy=True)
    
